In [1]:
%pylab inline
import pandas as pd
import seaborn
import numpy as np
import matplotlib.pyplot as plt
import scipy
import cv2
import nltk
import string
from collections import defaultdict
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import sys
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold

In [3]:
mile3_df_v1 = pd.read_json(r'C:\Users\HP\Documents\cmpt 459 Kaggle\two_sigma\final_train_df_m3.json')

In [4]:
mile3_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48329 entries, 0 to 48328
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bathrooms          48329 non-null  float64
 1   bedrooms           48329 non-null  int64  
 2   building_id        48329 non-null  object 
 3   created            48329 non-null  object 
 4   description        48329 non-null  object 
 5   display_address    48329 non-null  object 
 6   features           48329 non-null  object 
 7   latitude           48329 non-null  float64
 8   listing_id         48329 non-null  int64  
 9   longitude          48329 non-null  float64
 10  manager_id         48329 non-null  object 
 11  photos             48329 non-null  object 
 12  price              48329 non-null  int64  
 13  street_address     48329 non-null  object 
 14  interest_level     48329 non-null  object 
 15  dt_distance        48329 non-null  float64
 16  pet_policy         483

In [4]:
v1_selected_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "pet_policy", "listing_type", 
                     "unit_features", "building_features", "apartment", "appliance", "bathroom", "bedroom",
                     "building", "contact", "floor", "high", "kitchen", "room", "num_photos", 
                     "description_len", "mall_distance", "dt_distance"]

In [6]:
X = mile3_df_v1[v1_selected_feats]
y = mile3_df_v1["interest_level"]
X_data = X.to_numpy()

In [7]:
#CV for stacking
####
####  Note: Try with X instead of X_data 
####
scores = []
train_scores = []
train_LogLoss = []
test_LogLoss = []
estimators = [('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
             ('svr', make_pipeline(SVC(C=1, kernel = 'sigmoid', degree = 5, max_iter=100, gamma='auto', probability=True)))] 
#                                LinearSVC(random_state=42)))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
cv = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in cv.split(X):
    X_train, X_test, y_train, y_test = X_data[train_index], X_data[test_index], y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))
    train_scores.append(clf.score(X_train, y_train))
    y_test_pred = clf.predict_proba(X_test)
    test_LogLoss.append(log_loss(y_test, y_test_pred))
    y_train_pred = clf.predict_proba(X_train)
    train_LogLoss.append(log_loss(y_train, y_train_pred))

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with Standar

In [8]:
print(scores)
print(train_scores)
print(train_LogLoss)
print(test_LogLoss)

[0.728946823918891, 0.7243947858472998, 0.7252224291330437, 0.7176701841506311, 0.7240558717020176]
[0.9713938390709463, 0.9709282776815042, 0.9723508263714663, 0.9732302201070792, 0.9714980343471964]
[0.27062630099543306, 0.27800597997307414, 0.2662921571824835, 0.2653646762528373, 0.2750274320037054]
[0.6280245717524602, 0.6367436532598597, 0.6374981323204416, 0.6462520972991784, 0.6318160320409921]


In [23]:
# clf.fit(X, y)

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with Standar

StackingClassifier(cv=None,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                     

In [24]:
df_test = pd.read_json(r'C:\Users\HP\Documents\cmpt 459 Kaggle\two_sigma\final_test_df_m3.json')
print(df_test.shape)

(74659, 45)


In [25]:
X_kaggle = df_test[v1_selected_feats]

In [26]:
y_kaggle = clf.predict_proba(X_kaggle)

In [29]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}

In [31]:
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [32]:
sub = pd.DataFrame()
sub["listing_id"] = df_test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_kaggle[:, labels2idx[label]]
sub.to_csv("submission_v1_1.0.csv", index=False)